In [16]:
import numpy as np
import datacube
%matplotlib inline
import matplotlib
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
import fiona
import pprint
from madmex.io.vector_db import VectorDb, load_segmentation_from_dataset
from django.contrib.gis.geos import Polygon
from madmex.models import PredictObject
from madmex.models import Model, PredictClassification, Tag
from django.contrib.gis.geos import Polygon
from madmex.util import chunk

In [ ]:
#name of model
name_model = 's2_fiona_test_model_29_-29'

In [ ]:
model_id = Model.objects.get(name=name_model).id

In [ ]:
#name of prediction

In [3]:
name_predict = 's2_fiona_predict'
predict_object_id = 1 #TODO: delete this and change it to a programatically way

In [14]:
pred_objects = PredictClassification.objects.filter(predict_object_id=predict_object_id,name=name_predict)


In [24]:
pred_objects[0].features_id

0

In [29]:
pred_objects[0].tag_id

17

In [17]:
seg = PredictObject.objects.filter(id=pred_objects[0].predict_object_id)

In [18]:
seg

<QuerySet [<PredictObject: PredictObject object (1)>]>

In [25]:
path = seg[0].path

In [ ]:
#write to shapefile

In [36]:
qs = Tag.objects.filter(scheme='madmex_31')
tag_mapping = {x.id:x.numeric_code for x in qs}
tag_id_list = [pred_objects[x].tag_id for x in range(0,len(pred_objects))]


In [48]:
labels=[tag_mapping[tag_id_list[x]] for x in range(0,len(tag_id_list))]

In [49]:
with fiona.open(path) as source:
    source_driver = source.driver
    source_crs = source.crs
    source_schema = source.schema

In [50]:
fc_schema = {'geometry': 'Polygon',
             'properties': {'code': 'int'}}

In [51]:
with fiona.open(path) as src:
    fc_pred = [(x[0]['geometry'], x[1]) for x in zip(src, labels)]

In [52]:
fc_pred_geojson = [{'geometry': x[0],
                      'properties': {'code': x[1]}, #why is necessary to transform to int ???
                      'type': 'feature'} for x in fc_pred]

In [53]:
pprint.pprint(fc_pred_geojson[0])

{'geometry': {'coordinates': [[(2427740.0, 1026160.0),
                               (2428250.0, 1026160.0),
                               (2428250.0, 1025650.0),
                               (2427740.0, 1025650.0),
                               (2427740.0, 1026160.0)]],
              'type': 'Polygon'},
 'properties': {'code': 28},
 'type': 'feature'}


In [54]:
with fiona.open('/shared_volume/tasks/2019/fiona_cloud_usage/fiona_usage_and_db_modified.shp','w', 
                driver=source_driver,
                crs=source_crs,
                schema=fc_schema) as dst:
    write = dst.write
    [write(feature) for feature in fc_pred_geojson]